In [ ]:
import requests 
import json
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
import sys
sys.path.append('../')
import src.cleaning_functions as cf

In [ ]:
load_dotenv()

In [ ]:
!pip3 install python-dotenv

In [ ]:
pd.set_option('display.max_columns', None) #con este parámetro puedo ver TODAS las columnas de un df.

### 1. Importo mi dataframe

In [ ]:
df= pd.read_csv("../data/Food and Calories.csv")

In [ ]:
df.head()

### Limpio mi data frame eliminando nulos y duplicados

In [ ]:
df.shape

In [ ]:
df.dropna(how="all", inplace=True)

In [ ]:
df.shape #no hay ninguna fila entera de NaN

In [ ]:
duplicates= df.drop_duplicates() #elimino los duplicados

In [ ]:
duplicates.shape

In [ ]:
food = duplicates["Food"].unique()

### Obtengo una lista con los valores únicos de la columna Food que serán los valores de mi dataframe que utilizaré con la información de la API con la que trabaje a continuación

In [ ]:
lista_food= list(food)

In [ ]:
type(lista_food)

In [ ]:
lista_food

In [ ]:
type(lista_food)

In [ ]:
len(lista_food)

### A continuación voy a aplicar la función para la lista completa de alimentos de mi dataframe

In [ ]:
api_id = os.getenv("app_id")
app_key = os.getenv("app_key")

In [ ]:
prueba2 = cf.call_api(api_id,app_key,lista_food)

In [ ]:
prueba2

In [ ]:
len(prueba2)

### Consigo un dataframe de la información que me ofrece la API

In [ ]:
df_api = json_normalize(prueba2)

In [ ]:
df_api.head()

In [ ]:
df_api.shape

### Voy a limpiar el dataframe obtenido de la API, para ello creo un dataframe nuevo con las columnas que me van a dar información relevante, además incluyo la columna de ingredientes de mi dataframe inicial.

In [ ]:
df_clean = df_api[["dietLabels","healthLabels","cautions","totalNutrients.FAT.quantity","totalNutrients.SUGAR.quantity","totalNutrients.PROCNT.quantity","totalNutrients.CHOLE.quantity","totalNutrients.CA.quantity","totalNutrients.VITC.quantity","totalNutrients.WATER.quantity","totalNutrients.SUGAR.added.quantity"]]
df_clean["Ingredient"] = lista_food

In [ ]:
df_clean.head()

In [ ]:
rename = {"totalNutrients.FAT.quantity": "FAT.Total lipid","totalNutrients.SUGAR.quantity":"Total SUGAR(g)","totalNutrients.PROCNT.quantity":"Total PROTEIN(g)","totalNutrients.CHOLE.quantity":"Cholesterol(mg)","totalNutrients.CA.quantity":"Calcium(mg)","totalNutrients.VITC.quantity":"VIT C(mg)","totalNutrients.WATER.quantity":"Whater quantity","totalNutrients.SUGAR.added.quantity":"Sugar added"}

In [ ]:
df_clean.rename(columns=rename, inplace=True)

In [ ]:
df_clean

In [ ]:
df_clean["healthLabels"][1]

In [ ]:
mergeado = df_clean.merge(duplicates, how="left", left_on = "Ingredient", right_on="Food")

In [ ]:
mergeado.head()

In [ ]:
mergeado.drop(columns=["Ingredient"], inplace=True)

In [ ]:
mergeado.head()

In [ ]:
mergeado = mergeado.reindex(columns=['Food','Serving','Calories','dietLabels','healthLabels','cautions','FAT.Total lipid','Total SUGAR(g)','Total PROTEIN(g)','Cholesterol(mg)','Calcium(mg)','VIT C(mg)','Whater quantity','Sugar added'])

In [ ]:
mergeado

In [ ]:
mergeado.to_csv("../data/Food_enriched.csv")